# Submodule 1: Introduction to Genome Sequencing and Assembly
--------
## Overview
In this submodule you will learn how DNA and RNA sequencing data is produced and how to reconstruct the genome of an organism from short DNA sequences generated by sequencing technologies. The process will be completed through an interactive markdown document and will use command-line bioinformatic tools. We will explore the key concepts and methodologies in genomics, including how to assemble a genome, how to assess its quality, how to annotate the genome, and how to perform comparative analyses.

<p align="center">
  <img src="images/diagram-WGS.png" width="50%"/>
</p>

Figure 1. Overview of the bioinformatic workflow for the four submodules in this tutorial.
### Learning Objectives:

- **Understand how sequencing data is generated**: Learn the techniques and technologies used to generate genomic sequencing data, including next-generation sequencing (NGS) and its impact on genome assembly.

- **Develop an understanding of core bioinformatic input/output formats in genomics**: Gain proficiency in handling common file formats like FASTQ, BAM, and GFF, and understand their role in processing, storing, and analyzing genomic sequencing data.

- **Acquire the skills to assemble raw sequencing reads into a draft genome**: Master the process of genome assembly, from quality control of sequencing data to using assembly tools for constructing a draft genome from raw reads.

## Background: How is sequencing data produced?

### What is Next-Generation Sequencing (NGS)?

Next-Generation Sequencing (NGS) is a high-throughput method that allows for rapid sequencing of DNA. Illumina remains the most widely used sequencing platform. It is accurate, inexpensive, and fast, providing up to 540 Gb of data in about a 1-2 days. Other long-read sequencing technologies (i.e. PacBio and Nanopore) are becoming widely used in the research community, and will be discussed in later bioinformatic modules.

Figure 2 outlines the basic workflow for sequencing a microbial genome. It begins with the **isolation of bacterial cultures**, which involves culturing bacteria from a sample on selective media to obtain a pure strain. Once a pure culture is established, **DNA extraction** is performed to isolate the genomic DNA from the bacterial cells.

For Illumina-based sequencing, DNA is **fragmented** into smaller pieces, and adapters and barcode indexes are added to each fragment to enable sequencing and sample multiplexing. This step is called **library preparation**. The prepared sequencing libraries are then attached to a specially coated surface on a microscope slide, forming dense clusters of identical DNA fragments ready for sequencing. In a process called **sequencing by synthesis**, fluorescently labeled nucleotides are incorporated into the clusters one at a time, using the complementary base pairing properties of DNA. Each incorporated nucleotide emits a distinct signal, allowing the sequencer to determine the DNA sequence in real time.


This technology enables millions of fragments to be sequenced simultaneously, significantly increasing throughput and reducing costs.


Isolate Bacteria            |  Extract DNA
:-------------------------:|:-------------------------:
![alt text](https://github.com/Joseph7e/HCGS-Genomics-Tutorial/blob/master/petri.jpg?raw=true)  |  <img src="https://www.cephamls.com/wp-content/uploads/2019/02/DNA-Extraction-Figure-3-22.jpg" width="420">

Prepare Library           |  Sequence DNA
:-------------------------:|:-------------------------:
<img src="https://jef.works//assets/blog/librarystructure.png" width="520">  |  <img src="https://github.com/Joseph7e/HCGS-Genomics-Tutorial/blob/master/hiseq.png?raw=true" width="320">

Figure 2. The process of sequencing an organisms DNA.


## Video Lesson: How next-generation sequencing works.

The following video describes the process of preparing an Illumina sequenicng library and how genome sequenicng with an Illumina sequencer works. We also compare and contrast this to other sequencing technologies (PacBio and Nanopore). Run the HTML code below to strat the video.

<div class="alert alert-block alert-info"><b>Tip</b>: To execute the code you can either press <b>ctrl-ENTER</b> when the cell is selected or by pressing the play button at the top of the notebook</div>

In [ ]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/p4vKJJlNTKA" frameborder="0" allowfullscreen></iframe>

## **Install required software**

Four main tools are required for for Submodule 1: **fastq-dump, fastqc, fastp, and spades**. As described in the README, all tools are preinstalled into a docker image for this tutorial, but we will demonstrate how to install these tools using __[Conda](https://docs.conda.io/en/latest/)__, an open-source package management system and environment management tool. Conda helps to easily install, update, and manage software dependencies across different platforms. When we install the tools we select specific versions to ensure consistent behavior and reproducibility of results across different environments and systems.

### List of software

| **Tool**       | **Description**                                                                                                                                                           |
|:---------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **sra-tools**      | fastq-dump is used to retrieve publicly available data from the NCBI Sequence Read Archive (SRA).                                                 |
| **fastqc**        | Used to assess the quality of raw sequencing data by generating visual reports on metrics like read length, GC content, and sequence duplication.|
| **fastp**   | Used for quality control and pre-processing of FASTQ files, including read trimming, adapter removal, and filtering out low-quality reads.                          |
| **spades**      | Used for genome assembly, creating contigs from high-quality, pre-processed sequencing reads.                  |  

<div class="alert alert-block alert-warning"> <b>Attention:</b> The code below is written in Bash. The <code>%%bash</code> at the beginning of the cell instructs the Jupyter notebook to execute the code as a Bash script when the cell is run. This is the same code you would run from a standard command-line terminal</a>. </div>

<div class="alert alert-block alert-info"><b>Tip</b>: To execute the code you can either press <b>ctrl-ENTER</b> when the cell is selected or by pressing the play button at the top of the notebook</div>

In [ ]:
%%bash

echo -e "\033[1mNote: All command-line tools are preinstalled in a docker image\033[0m"
echo -e "\033[1mNote: The command below provides an alternative means to aquire the needed tools using conda, but is not run by default\033[0m"
echo "------------------------------------------------------------------------------"
# Install tools using mamba (a conda alternative) with specified versions

echo "mamba install --channel bioconda \
    python=3.9 \
    sra-tools=3.1.1 \
    fastqc=0.11.8 \
    fastp=0.23.4 \
    spades=4.0.0 \
    -y > /dev/null 2>&1"

echo "------------------------------------------------------------------------------"
echo -e "\033[1mInstallation of sra-tools, fastqc, fastp, and spades complete.\nDisplaying versions:\033[0m"

# Confirm installation by checking versions
fastq-dump --version
fastqc --version
fastp --version
spades.py --version

<div class= "alert alert-block alert-info"><b>Tip</b>: use <code>\</code> to break a long command into multiple lines</div>

<div class= "alert alert-block alert-info"><b>Tip</b>: use <code>/dev/null 2>&1</code> to write standard error and standard output to <i>/dev/null</i>, effectively discarding them so your screen doesn't get flooded with text.</div></div>

## **Starting data**
The data used for this module is described in a manuscript comparing phenotypic and whole-genome sequencing derived AMR profiles ([Painset et al. 2020](https://pubmed.ncbi.nlm.nih.gov/31943013/)) and is available under the SRA accession [SRR10056829](https://www.ncbi.nlm.nih.gov/sra/?term=PRJNA505131). Genomic DNA was extracted, fragmented and prepared for multiplexing using a **Nextera XT DNA Sample Preperation Kits**. Paired-end sequencing data was produced on an **Illumina HiSeq 2500** platform.

The manuscript includes sequencing read datasets for 528 isolates of *Campylobacter* spp. (452 *C. jejuni* and 76 *C. coli*) from human (494), food (21) and environmental (2) sources. We will start by walking through the process on just a single dataset. We will later expand this to include a larger set of this data in submodule 3.

In [ ]:
%%bash

# Capture the SRA accession in a variable 
accession=SRR10056829

# Download data from the SRA using the variable from above 

# prefetching downloads metadata of the SRA records (5.629s)
echo "Prefetching metadata from the NCBI SRA"
prefetch -v $accession > /dev/null 2>&1

# fastq-dump downlaods the reads and compresses them using standard gzip compression. (1m33.96s)
echo "Downloading the reads and compressing them using standard gzip compression"
fastq-dump --outdir raw-reads --gzip --split-files "$accession"/"$accession".sra > /dev/null 2>&1

# Remove the prefetch directory
rm -r $accession

# rename files so this tutorial works with any data
mv raw-reads/SRR10056829_1.fastq.gz raw-reads/reads_1.fastq.gz
mv raw-reads/SRR10056829_2.fastq.gz raw-reads/reads_2.fastq.gz
echo "Process Complete"

<div class= "alert alert-block alert-info"><b>Tip</b>: Assigning certain values to variables (like the accession above) allows you to easily adjust input to a set of commands and reduces the amount of typing for long or complicated file names</div>

## FASTQ file format
The data we just downloaded are in **FASTQ** file format. Let's examine this data and file format.

Let's run three sections of code to learn about the FASTQ file format. Please also see a detailed description of FASTQ files in the NIGMS module [Fundementals of Bionformatics](https://github.com/NIGMS/Fundamentals-of-Bioinformatics)

In [ ]:
%%bash

# If the above code worked you should have two FASTQ files in a directory called raw-reads
ls raw-reads/

<div class="alert alert-block alert-warning"> <b>Attention:</b> Notice the the files above have the .gz extension. FASTQ data is commonly gzip compressed to save sapce. Most bioinformatic tools accept or require gzipped FASTQs. </div>

In [ ]:
%%bash

# define variable names for reads
forward=raw-reads/reads_1.fastq.gz
reverse=raw-reads/reads_2.fastq.gz

# view the first four lines
echo "-------- FASTQ READ ---------"
zcat $forward | head -n 4
echo "-----------------------------"

### Explanation:
The above code should have displayed the first four lines of the FASTQ file. **Each sequencing read in a FASTQ formatted file is made up of four lines**. 

The length of the sequencing reads can vary depending on the type of sequencing instrument and flow cell used, but they should be consistent across the raw data. In our example dataset the reads are 100 bps in length. Illumina sequencers produce reads with length lengths randing between 50 and 300 bps. Run the following code to print the length of the reads.

In [ ]:
%%bash
zcat raw-reads/reads_1.fastq.gz | head -n 2 | tail -n 1 | wc -c

<div class= "alert alert-block alert-info"><b>Tip</b>: The number above is one higher than the actual read length, this is because of the newline charcater at the end of each line that counted with the wc command</div>

## Do we have enough data? 

Most bacterial genomes consist of a single, circular chromosome, although some bacteria may have linear chromosomes or multiple chromosomes. In addition to the main chromosome, plasmids (small circular DNA molecules) often coexist, carrying additional genes. Compared to eukaryotes, bacterial genomes are typically much smaller, ranging from approximately 0.5 to 10 megabases (Mb) in size. For instance, Escherichia coli has a genome size of about 4.6 Mb. This genome can be represented as a string of 4.6 million nucleotides—comprising "A"s, "T"s, "C"s, and "G"s.

| Organism                                  | Genome Size (Mb) |
|------------------------------------------|------------------|
| *Mycoplasma genitalium* (Bacteria)       | 0.58             |
| *Escherichia coli* (Bacteria)            | 4.6              |
| *Saccharomyces cerevisiae* (Yeast)       | 12.1             |
| *Streptomyces coelicolor* (Bacteria)     | 8.7              |
| *Caenorhabditis elegans* (Nematode)      | 100              |
| *Arabidopsis thaliana* (Thale Cress)     | 135              |
| *Drosophila melanogaster* (Fruit Fly)    | 139              |
| *Zea mays* (Maize)                        | 2,300            |
| *Mus musculus* (Mouse)                   | 2,700            |
| *Homo sapiens* (Human)                   | 3,200            |


### What is sequencing 'coverage'?
**Breadth of coverage** refers to the percentage of the genome that has been sequenced at least once. It tells you how much of the genome has any sequencing data aligned to it, but it doesn’t indicate how many times each base has been sequenced. High breadth of coverage means most or all regions of the genome are represented in the sequencing data.

**Depth of coverage**, on the other hand, refers to the average number of times each base in the genome is sequenced, often represented as "X" (e.g., 30x depth). Depth is important for accuracy, as higher depth ensures that individual bases are sequenced multiple times, improving confidence in variant calling and reducing sequencing errors.

When researchers say *coverage* they usually mean *depth of coverage*.


The equation for genome depth of coverage is:

$$
\text{Coverage} = \frac{N \times L}{G}
$$

Where:
- \( N \) = Number of reads
- \( L \) = Read length
- \( G \) = Genome size


The total number of bases sequenced, the numerator in the above equation, is also called the total **throughput** for the sample.


### How can we determine our total throughput and coverage?

Lets do the math based on the data we have so far, we know the read length, and we can use a best guess at the genome size as 5 Mb for now. So lets count our number of reads.

In [ ]:
%%bash

# counting the lines and dividing by 4. Remember each read entry is exactly four lines long. These numbers should match.
echo "The total number of lines in the fastq file:"
zcat raw-reads/*_1.fastq.gz | wc -l

In [ ]:
%%bash

# we can do this calculation from the terminal with echo and bc (bc is the terminal calculator)
num_reads=6365120/4
read_length=100
# adjust this number to the organisms genome size.
genome_size=5000000

# calculate our total throughput
echo "Total number of base pairs sequenced:"
#throughput=$(echo $num_reads " * " $read_length " * 2" | bc)
throughput=$(awk "BEGIN {print $num_reads * $read_length * 2}")
echo $throughput

# calculate our estimated coverage
echo "The estimated depth of coverage for this genome dataset:"
#coverage=$($throughput " / " $genome_size | bc)
coverage=$(awk "BEGIN {print $throughput / $genome_size}")
echo $coverage


### Explanation

Our estimated coverage, assuming a 5 Mb genome, is 63X. This means we are estimated to have sequenced every base in the genome approximately 63 times.

**Coverage and De Novo Assembly:**
For de novo genome assembly, coverage is a critical factor. Typically, **30X to 50X coverage** is recommended for most bacterial genomes, as it provides enough data to accurately assemble the genome without over-assembling, which could lead to potential issues. However, the required coverage can vary depending on factors such as the organism’s genome size, complexity, and the sequencing platform used.

- **Bacterial genomes**: 30X to 50X is generally sufficient.
- **Eukaryotic genomes**: Larger genomes may require higher coverage, often in the range of 50X to 100X or more, depending on the complexity and repetitive regions.

Higher coverage helps ensure the assembly is accurate and complete, reducing the chances of gaps or errors. However, after a certain point, increasing coverage does not significantly improve the assembly quality and only increases sequencing costs.


## Process 1: Quality Assessment with FASTQC
- **Program**: **FASTQC**
- **Citation**: *Andrews, S. (2010). FastQC: A Quality Control Tool for High Throughput Sequence Data.*
- **Manual**: https://www.bioinformatics.babraham.ac.uk/projects/fastqc/
- **FASTQC explained**: https://www.bioinformatics.babraham.ac.uk/projects/fastqc/Help/3%20Analysis%20Modules/


FastQC is a program to summarize read qualities and base composition. Since we have millions of reads there is no practical way to do this by hand. We call the program to parse through the fastq files and determine some basic statistics about the data. The input to the program is one or more fastq file(s) and the output is an html file with several figures. The link above describes what each of the output figures are describing. I mainly focus on the first graph which visualizes our average read qualities and the last figure which shows the adapter content. Note that this program does not do anything to your data, it merely reads it and writes a report.

In [ ]:
%%bash
# make a directory to store the output
mkdir -p output-fastqc

# run the program on the two read files (R1 and R2).
echo "RUNNING FASTQC"
fastqc raw-reads/*.fastq.gz -o output-fastqc --threads 24 > /dev/null 2>&1
echo "FASTQC analysis complete, output files:"

# the resulting folder should contain a zipped archive and an html file, we can ignore the zipped archive which is redundant.
ls output-fastqc/

### FASTQC output

The code below will open one of the interactive HTML output files produced by FASTQC.   

I will highlight a few observations about this data. FASTQC is a standard tool described in detail elsewhere, including in other NIGMS modules.  Take a moment to open the link above [FASTQC explained](https://www.bioinformatics.babraham.ac.uk/projects/fastqc/Help/3%20Analysis%20Modules/) and look at each figure in your report. The [manual](https://www.bioinformatics.babraham.ac.uk/projects/fastqc/) also provides links to examples of good and bad reports for comparison to your data.  

- In our dataset we see a warning (**X**) on the *Per base sequence content*. This figure displays the proportion of each nucleotide (A, T, C, and G) at every position across all reads in the FASTQ file. During library preparation, the DNA underwent fragmentation, typically performed either by physical shearing or enzymatic digestion. Ideally, this process should be truly random. However, we observe a bias in our data, with an overrepresentation of 'T' nucleotides at the beginning of the reads. It appears the DNA may have preferentially fragmented at thymine-rich regions.
- We also see a warning (**X**) on the *Sequence Duplication Levels*. FASTQC calculates how often each sequence appears in the dataset and generates a plot showing the distribution of duplication across the library. In a complex and diverse library, most sequences are expected to occur only once. While a low level of duplication can sometimes reflect deep coverage of a target region, a high level of duplication more often indicates technical bias, such as PCR over-amplification or limited library complexity. Identifying excessive duplication is important for assessing library quality and determining whether further optimization of the library preparation process is needed.  
- I would like to take a moment to look at the *Per sequence GC content*. This figure displays the GC content calculated for each read in the FASTQ file and compares the observed distribution to a modeled normal distribution based on an ideal dataset. Each organism has a unique GC content signature, and we’ll explore this in more detail in Submodule 2. Deviations from the expected distribution in this figure may indicate contamination, sequencing bias, or the presence of sequences from organisms with differing GC content.
- Also see the *Adapter Content* module, which identifies whether any standard adapter sequences are present in the reads. Adapters are added to the ends of DNA fragments during library preparation to enable sequencing. However, if a DNA fragment is shorter than the sequencing read length (100 bp in our case), the sequencing instrument may read through the fragment and into the adapter sequence. When this happens, it’s important to detect and remove these artificial sequences before proceeding with downstream analyses, such as read alignment or assembly, to avoid introducing bias or errors.

There are warnings in this dataset, but they are not cause for concern. FASTQC does not alter the data in any way; instead, it provides helpful diagnostics and observations to be aware of as you move forward with your analysis.  

In [1]:
## view HTML report
from IPython.display import IFrame
IFrame('output-fastqc/reads_1_fastqc.html', width=1000, height=550)

## Process 2: Quality and adapter trimming with FASTP

- **Program**: **fastp** - An Ultra-Fast All-in-One FASTQ Preprocessor
- **Citation**: *Chen, S. (2023). Ultrafast one‐pass FASTQ data preprocessing, quality control, and deduplication using fastp. Imeta, 2(2), e107.*
- **Manuscript**: [https://pubmed.ncbi.nlm.nih.gov/30423086/](https://pubmed.ncbi.nlm.nih.gov/30423086/)
- **Manual**: [fastp GitHub Repository](https://github.com/OpenGene/fastp)

Description:
**fastp** is an all-in-one tool designed for preprocessing and quality control of high-throughput sequencing data. It performs several critical tasks, including base correction, adapter trimming, read filtering, and quality score recalibration. Additionally, it provides deduplication to remove PCR duplicates and offers comprehensive quality control through detailed reports and visualizations. Its efficiency allows it to handle large datasets rapidly, making it ideal for processing next-generation sequencing (NGS) data in various formats like FASTQ. Alternative tools for similar purposes include **Trimmomatic**, **Cutadapt**, and **FASTX-Toolkit**, which offer some of the same functionalities but may differ in speed, ease of use, and specific features.

In [ ]:
%%bash

# make a directory to store the output
mkdir output-fastp

# define input variables
forward=raw-reads/reads_1.fastq.gz
reverse=raw-reads/reads_2.fastq.gz

# run the program
fastp --in1 $forward --in2 $reverse --out1 output-fastp/trimmed_1.fastq.gz --out2 output-fastp/trimmed_2.fastq.gz

# the resulting folder should contain new trimmed read files
ls output-fastp/

### Explanation of fastp Results

The **fastp** results printed above show the quality of the sequencing data both before and after filtering. These results provide insight into the number of reads, base quality, and the effectiveness of filtering steps like adapter trimming and quality control.

- **<u>Before filtering</u>**:
  - **Read1** and **Read2** both start with 1,591,280 reads, with high overall quality scores. Over 99% of bases have a quality score (Q20) above 20, and about 96% of bases have a quality score (Q30) above 30, indicating good overall quality in the raw data.

- **<u>After filtering</u>**:
  - The total number of reads decreases slightly to 1,568,979 for both **Read1** and **Read2**. Despite this reduction, the quality improves, with over 99% of bases achieving Q20 and over 96% of bases achieving Q30 for **Read1**, and over 98% for **Read2**. This shows that filtering removed low-quality reads while maintaining high-quality data.

- **<u>Filtering summary</u>**:
  - A total of 31,379,958 reads passed the quality filter, while 43,808 reads failed due to low quality. A small number (794) failed due to the presence of too many 'N' bases, and no reads were discarded due to being too short.
  - **Adapter trimming** removed 5,320,114 bases across both reads, which improves the accuracy of downstream analysis.

- **<u>Duplication rate</u>**: 
  - The duplication rate is low at 2.70%, meaning most reads are unique and useful for assembly and analysis.

- **<u>Insert size</u>**: 
  - The insert size peak is 168, which reflects the typical distance between paired-end reads, providing valuable information about the sequencing library.

These results suggest that **fastp** was effective in cleaning the raw sequencing data by removing low-quality reads and trimming adapter sequences, resulting in improved data quality suitable for downstream analyses.

## Process 3: Genome Assembly

- Program: **SPAdes - St. Petersburg genome assembler**
- Citation: *Bankevich, A., Nurk, S., Antipov, D., Gurevich, A. A., Dvorkin, M., Kulikov, A. S., ... & Pevzner, P. A. (2012). SPAdes: a new genome assembly algorithm and its applications to single-cell sequencing. Journal of computational biology, 19(5), 455-477.*
- Manuscript: [https://pubmed.ncbi.nlm.nih.gov/22506599/](https://pubmed.ncbi.nlm.nih.gov/22506599/)
- Manual: https://ablab.github.io/spades/  

SPAdes (St. Petersburg genome assembler) is a genome assembly algorithm which was designed for single cell and multi-cells bacterial data sets. SPAdes works with Ion Torrent, PacBio, Oxford Nanopore, and Illumina paired-end, mate-pairs and single-end reads.  

Adjust the --threads option to match the number of CPU cores available on the machine type you selected during setup. If you followed the recommended configuration, you can set --threads 32. With this setting, the process should take less than 10 minutes to complete.

<div class="alert alert-block alert-info"><b>Tip</b>: I recommend running the code below, then watching the following video tutorial or reading more while it runs.</div>

In [ ]:
%%bash
## Assemble trimmed reads using the SPAdes assembler.

# define input
forward=output-fastp/trimmed_1.fastq.gz
reverse=output-fastp/trimmed_2.fastq.gz

# assemble reads with SPAdes
echo "RUNNING THE SPADES PIPELINE, PLEASE REVIEW MATERIALS BELOW WHILE IT RUNS"
spades.py -1 $forward -2 $reverse -o output-spades --threads 32 > logfile_spades.txt 2>&1
echo "SPADES assembly complete"

## Video Lesson: How genome assembly of short reads works.

The following video describes the process in detail. Run the HTML code below to start the video.

In [ ]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/ZmF6QROPlTU" frameborder="0" allowfullscreen></iframe>

## Video Recap

### De Bruijn Graphs in Genome Assembly

A **de Bruijn graph** is a structure used in genome assembly to represent sequences using **k-mers**. Here’s how it works:

1. **K-mers**: These are short sequences of length *k* derived from a longer DNA sequence. For example, from the sequence "AGCT", the 2-mers are "AG", "GC", and "CT".

2. **Graph Construction**: Each k-mer becomes a vertex in the graph. An edge is drawn from k-mer **A** to k-mer **B** if the last *k-1* nucleotides of **A** match the first *k-1* nucleotides of **B**. This creates a directed graph where paths through the vertices represent potential sequences in the genome.

3. **Contigs and Scaffolds**:
   - **Contigs**: These are contiguous sequences formed by traversing paths in the graph.
   - **Scaffolds**: Longer constructs that may combine multiple contigs, providing additional structure to the genome assembly.

De Bruijn graphs enable efficient handling of sequencing data, especially for complex genomes with repetitive sequences.
  
  
### Key challenges for genome assembly:
**<u>Intrinsic Challenges:</u>**
1. **Heterozygosity:** The alleles of a gene are not he same, yet we force them into a single consensus sequence.
2. **Paralogy vs. Alleleism**: Genes come from other genes by a process of duplication. This results in two or more similar genes in an organism. There are two alleles in a diploid organism that are very similiar. How do you tell a duplicated gene from alleles of a gene?
3. **Sequence complexity**: Simple sequence repeats (SSR), large scale repeats like transposable elements (TEs).

**<u>Extrinsic Challenges:</u>**
1. **Quality of DNA sequences (sequencing errors):** Each sequencing technology has specific patterns of error.
2. **Length of DNA sequence reads**: Shorter reads are less likely to be unique or to include unique K-mers (see below).
3. **Coverage**: Depth of coverage is a random process at best. Consequently some regions of the genome will have low levels of coverage.
4. **Memory intensive**: Inherently requires large amounts of RAM for assebly and storage of input and output.




## Conclusion and wrap up

End of submodule 1.  After learning about how genome sequencing works (and where it can go wrong) we went through standard processing steps (FASTQC and FASTP) and ultimetly ran a genome assembly tool. The final output from spades represents our draft genome sequence.

Be sure to shut down your instance. We will run one more bit of code to copy our genome file for use in submodule 2.

In [ ]:
%%bash
# did the assembler finish?
ls output-spades/

<div class="alert alert-block alert-info"><b>Status Check</b>: If the assembly finished successfully you will see the file called contigs.fasta above. This is our genome assembly. Be sure to run the next set of code to prepare for submodule 02.</div>

In [ ]:
%%bash
# make a copy of the genome sequence in a new directory. This data will be used for Submodule 2.
mkdir -p assembled-genome
cp output-spades/contigs.fasta assembled-genome/genome.fasta


# cleanup other files
mkdir -p submodule01_data
mv output-spades output-fastqc output-fastp submodule01_data
mv fastp.json fastp.html logfile_spades.txt submodule01_data

<div class="alert alert-block alert-warning"> <b>Attention Before You Proceed:</b>  If you're not moving on immediately to the next submodule, be sure to shutdown your instance. </div>